In [1]:
import numpy as np
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from dgl import model_zoo,DGLGraph
import math
import networkx as nx
import matplotlib.pyplot as plt
from utils_mpnn import Meter, set_random_seed, collate, EarlyStopping, load_model,load_brl_dataset,regress,run_a_train_epoch,run_an_eval_epoch
import argparse
# from sklearn import svm, datasets
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import plot_confusion_matrix
import seaborn as sn
import pandas as pd
from sklearn import metrics
import collections

import torch.nn as nn
import torch.nn.functional as F


# define NN architecture
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        
        self.fc1 = nn.Linear(6*7+7, 2048)#(6*7, 4096)
        self.fc2 = nn.Linear(2048,1024)#(4096,1024)
        self.fc3 = nn.Linear(1024,2048)#(1024,4096)
        self.fc4 = nn.Linear(2048,359*7)#(4096,359*7)
        self.droput = nn.Dropout(0.2)
        
    def forward(self,x):
        #x = x.view(-1,6*7)
        x = F.relu(self.fc1(x))
        #x = self.droput(x)
        x = F.relu(self.fc2(x))
        #x = self.droput(x)
        x = F.relu(self.fc3(x))
        #x = self.droput(x)
        x = self.fc4(x)
        return x
# initialize the NN
model = Net()

loss_fn =nn.L1Loss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
#stopper = EarlyStopping(mode='lower', patience=500, filename='model_saved/kld/early_stop.pth')
args={};args['device']=torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(args['device'])

class MyDataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = torch.from_numpy(data).float()
        self.target = torch.from_numpy(target).float()
        self.transform = transform
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target[index]
        if self.transform:
            x = self.transform(x)
        return x, y
    def __len__(self):
        return len(self.data)

data_path_x='data/data_mlp/';data_path_label='data/label_disp/'
mode='train'
dataset = MyDataset(np.load(data_path_x+'features_'+mode+'.npz')['arr_0'], np.load(data_path_label+'label_'+mode+'.npz')['arr_0'])
train_loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=True,pin_memory=True)
mode='val'
dataset = MyDataset(np.load(data_path_x+'features_'+mode+'.npz')['arr_0'], np.load(data_path_label+'label_'+mode+'.npz')['arr_0'])
val_loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=False,pin_memory=True)
mode='test'
dataset = MyDataset(np.load(data_path_x+'features_'+mode+'.npz')['arr_0'], np.load(data_path_label+'label_'+mode+'.npz')['arr_0'])
test_loader = torch.utils.data.DataLoader(dataset, batch_size=8, shuffle=False,pin_memory=True)

lr_flg=False

Using backend: pytorch


In [ ]:
lr_flg=True
if lr_flg:
    for param_group in optimizer.param_groups: print(param_group['lr'])
    for param_group in optimizer.param_groups: param_group['lr'] = param_group['lr']*0.1
    for param_group in optimizer.param_groups: print(param_group['lr'])

#number of epochs to train the model
n_epochs = 5000
# initialize tracker for minimum validation loss
valid_loss_min = np.Inf  # set initial "min" to infinity
for epoch in range(n_epochs):
    # monitor losses
    train_loss = 0
    valid_loss = 0
    # train the model #
    model.train() # prep model for training
    for data,label in train_loader:
        data = data.to(args['device'])
        label = label.to(args['device'])
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        output = output.to(args['device'])
        # calculate the loss
        loss = (loss_fn(output,label).float()).mean()
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item() * data.size(0)
         
    # validate the model #
    model.eval()  # prep model for evaluation
    for data,label in val_loader:
        data = data.to(args['device'])
        label = label.to(args['device'])
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        output = output.to(args['device'])
        # calculate the loss
        loss = (loss_fn(output,label).float()).mean()
        # update running validation loss 
        valid_loss = loss.item() * data.size(0)
    
    # print training/validation statistics # calculate average loss over an epoch
    train_loss = train_loss / len(train_loader.sampler)
    valid_loss = valid_loss / len(val_loader.sampler)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch+1, train_loss,valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        torch.save(model.state_dict(), 'model_saved/model.pt')
        valid_loss_min = valid_loss
        
lr_flg=True

In [ ]:
# initialize lists to monitor test loss and accuracy
pred=[];actual=[]
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
model.eval() # prep model for evaluation
for data, label in test_loader:
    data = data.to(args['device'])
    label = label.to(args['device'])
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    output = output.to(args['device'])
    # calculate the loss
    loss = (loss_fn(output,label).float()).mean()
    actual.append(label.cpu().detach().numpy())
    pred.append(output.cpu().detach().numpy())
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
# calculate and print avg test loss
test_loss = test_loss/len(test_loader.sampler)
print('Test Loss: {:.6f}\n'.format(test_loss))

pred=np.concatenate(pred,axis=0)
actual=np.concatenate(actual,axis=0)
print(pred.shape,actual.shape)
pred=pred.reshape((-1,359,7))
actual=actual.reshape((-1,359,7))
m_list=np.load('data/m_list.npy')
std_list=np.load('data/std_list.npy')
for i in range(7):
    pred[...,i]=pred[...,i]*std_list[i]+m_list[i]
    actual[...,i]=actual[...,i]*std_list[i]+m_list[i]
    
for i in range(7):
    print('==========')
    y_t=actual[...,i]
    y_p=pred[...,i]
    
    print(y_t.shape,y_p.shape)#(
    import sklearn.metrics as metrics
    mae = metrics.mean_absolute_error(y_t, y_p)
    mse = metrics.mean_squared_error(y_t, y_p)
    rmse=np.sqrt(mse)
    print('mae', mae, '| rmse:',rmse)
    print('actual mean', np.mean(y_t),'| pred mean',np.mean(y_p))
    print(np.corrcoef(y_t.flatten(),y_p.flatten()))
    print('rmse/range',rmse/(np.max(y_t)-np.min(y_t)))
    print('mape',np.mean(np.abs(y_t-y_p)/y_t))
    iqr= np.subtract(*np.percentile(y_t, [75, 25]))
    print('rmse/iqr',rmse/iqr)
    print('rmse/mean',rmse/np.mean(y_t))
    print('actual min max', np.min(y_t),np.max(y_t))
    print('pred min max', np.min(y_p),np.max(y_p))
    
for i in range(7):
    y_t=actual[...,i]
    y_p=pred[...,i]
    
    plt.plot(y_t.flatten(),y_p.flatten(),'.',color='darkblue',alpha=0.1)
    plt.ylim(np.min([y_p,y_t]), np.max([y_p,y_t]))
    plt.xlim(np.min([y_p,y_t]), np.max([y_p,y_t]))
    plt.xlabel('actual')
    plt.ylabel('pred')
    plt.grid()
    #plt.savefig('results/testset2.png')
    plt.show()

In [18]:
pred=[];actual=[]
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
model.eval() # prep model for evaluation
for data, label in test_loader:
    data = data.to(args['device'])
    label = label.to(args['device'])
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
    output = output.to(args['device'])
    # calculate the loss
    loss = (loss_fn(output,label).float()).mean()
    actual.append(label.cpu().detach().numpy())
    pred.append(output.cpu().detach().numpy())
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
# calculate and print avg test loss
test_loss = test_loss/len(test_loader.sampler)
print('Test Loss: {:.6f}\n'.format(test_loss))

pred=np.concatenate(pred,axis=0)
actual=np.concatenate(actual,axis=0)
pred=pred.reshape((-1,359,7))
actual=actual.reshape((-1,359,7))
print(pred.shape,actual.shape)

Test Loss: 0.004856

(237, 359, 7) (237, 359, 7)


In [45]:
mlp_output=np.load('data/data_mpnn_6/features_test.npz')['arr_0']
mlp_output=np.concatenate((pred,mlp_output),axis=1);print(mlp_output.shape)

test_loc=np.load('data/data_mlp/features_test_loc.npz')['arr_0']
mlp_output=np.concatenate((mlp_output,test_loc),axis=-1);print(mlp_output.shape)

np.save('data/mlp_output/mlp_output',mlp_output)

(237, 365, 7)
(237, 365, 10)


In [50]:
np.savez_compressed('data/mlp_output/features_test',mlp_output)